In [1]:
from pyspark.sql import SparkSession,Row

In [2]:
spark = SparkSession.builder.appName("test").master("local[*]").getOrCreate()

In [3]:
header =Row("ID", "Name", "gender", "Salary", "Department")

In [4]:
employee_df = spark.sparkContext.parallelize([
    header(1, "Deva", "Male", 5000, "Sales"), 
    header(2, "Jugnu", "Female", 6000, "HR"), 
    header(3, "Kavita", "Female", 7500, "IT"), 
    header(4, "Vikram", "Male", 6500, "Marketing"), 
    header(5, "Shabana", "Female", 5500, "Finance"), 
    header(6, "Shantilal", "Male", 8000, "Sales"), 
    header(7, "Vinod","Male", 7200, "HR"), 
    header(8, "Vimla", "Female", 6600, "IT"), 
    header(9, "Jasmin", "Female", 5400, "Marketing"), 
    header(10, "Lovely", "Female", 6300, "Finance"), 
    header(11, "Mohan", "Male", 5700, "Sales"), 
    header(12, "Purvish", "Male", 7000, "HR"), 
    header(13, "Jinat", "Female", 7100, "IT"), 
    header(14, "Eva", "Female", 6800,"Marketing"), 
    header(15, "Jitendra", "Male", 5000, "Finance"), 
    header(15, "Rajkumar", "Male", 4500, "Finance"), 
    header(15, "Satish", "Male", 4500, "Finance"), 
    header(15, "Himmat", "Male", 3500, "Finance")]).toDF()

In [7]:
from pyspark.sql.window import Window 
from pyspark.sql.functions import desc_nulls_last 

In [8]:
genderPartitionedSpec = Window.partitionBy("gender").orderBy(desc_nulls_last("Salary")) 

In [9]:
from pyspark.sql.functions import row_number 

In [10]:
employee_df.withColumn("rowNumber", row_number().over(genderPartitionedSpec)).show()

+---+---------+------+------+----------+---------+
| ID|     Name|gender|Salary|Department|rowNumber|
+---+---------+------+------+----------+---------+
|  3|   Kavita|Female|  7500|        IT|        1|
| 13|    Jinat|Female|  7100|        IT|        2|
| 14|      Eva|Female|  6800| Marketing|        3|
|  8|    Vimla|Female|  6600|        IT|        4|
| 10|   Lovely|Female|  6300|   Finance|        5|
|  2|    Jugnu|Female|  6000|        HR|        6|
|  5|  Shabana|Female|  5500|   Finance|        7|
|  9|   Jasmin|Female|  5400| Marketing|        8|
|  6|Shantilal|  Male|  8000|     Sales|        1|
|  7|    Vinod|  Male|  7200|        HR|        2|
| 12|  Purvish|  Male|  7000|        HR|        3|
|  4|   Vikram|  Male|  6500| Marketing|        4|
| 11|    Mohan|  Male|  5700|     Sales|        5|
|  1|     Deva|  Male|  5000|     Sales|        6|
| 15| Jitendra|  Male|  5000|   Finance|        7|
| 15| Rajkumar|  Male|  4500|   Finance|        8|
| 15|   Satish|  Male|  4500|  

In [12]:
#Select ntile (Various percentire) 
#If we divide salary in 3 quartile than in which quartile it fall 
from pyspark.sql.functions import ntile 

In [13]:
employee_df.select("*", ntile(3).over(genderPartitionedSpec).alias("ntile")).show() 

+---+---------+------+------+----------+-----+
| ID|     Name|gender|Salary|Department|ntile|
+---+---------+------+------+----------+-----+
|  3|   Kavita|Female|  7500|        IT|    1|
| 13|    Jinat|Female|  7100|        IT|    1|
| 14|      Eva|Female|  6800| Marketing|    1|
|  8|    Vimla|Female|  6600|        IT|    2|
| 10|   Lovely|Female|  6300|   Finance|    2|
|  2|    Jugnu|Female|  6000|        HR|    2|
|  5|  Shabana|Female|  5500|   Finance|    3|
|  9|   Jasmin|Female|  5400| Marketing|    3|
|  6|Shantilal|  Male|  8000|     Sales|    1|
|  7|    Vinod|  Male|  7200|        HR|    1|
| 12|  Purvish|  Male|  7000|        HR|    1|
|  4|   Vikram|  Male|  6500| Marketing|    1|
| 11|    Mohan|  Male|  5700|     Sales|    2|
|  1|     Deva|  Male|  5000|     Sales|    2|
| 15| Jitendra|  Male|  5000|   Finance|    2|
| 15| Rajkumar|  Male|  4500|   Finance|    3|
| 15|   Satish|  Male|  4500|   Finance|    3|
| 15|   Himmat|  Male|  3500|   Finance|    3|
+---+--------

In [14]:
#Divide with 25% and see in which 25%, employee salary false 
employee_df.select("*", ntile(4).over(genderPartitionedSpec).alias("ntile")).show() 

+---+---------+------+------+----------+-----+
| ID|     Name|gender|Salary|Department|ntile|
+---+---------+------+------+----------+-----+
|  3|   Kavita|Female|  7500|        IT|    1|
| 13|    Jinat|Female|  7100|        IT|    1|
| 14|      Eva|Female|  6800| Marketing|    2|
|  8|    Vimla|Female|  6600|        IT|    2|
| 10|   Lovely|Female|  6300|   Finance|    3|
|  2|    Jugnu|Female|  6000|        HR|    3|
|  5|  Shabana|Female|  5500|   Finance|    4|
|  9|   Jasmin|Female|  5400| Marketing|    4|
|  6|Shantilal|  Male|  8000|     Sales|    1|
|  7|    Vinod|  Male|  7200|        HR|    1|
| 12|  Purvish|  Male|  7000|        HR|    1|
|  4|   Vikram|  Male|  6500| Marketing|    2|
| 11|    Mohan|  Male|  5700|     Sales|    2|
|  1|     Deva|  Male|  5000|     Sales|    2|
| 15| Jitendra|  Male|  5000|   Finance|    3|
| 15| Rajkumar|  Male|  4500|   Finance|    3|
| 15|   Satish|  Male|  4500|   Finance|    4|
| 15|   Himmat|  Male|  3500|   Finance|    4|
+---+--------